In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import folium.features
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from useful_methods.ihm.maps.mapUtils import *
from useful_methods.neighbours_delaunay.simple_criteria import *
from useful_methods.neighbours_delaunay.enhanced_criteria import *
from useful_methods.data_processing import extract_data
from useful_methods.neighbours_delaunay.graphs import delaunay_graph
from city.city_utils import city_detection


In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')
providers = df['nom_op'].unique()
regions = df['nom_reg'].unique()
technologies = pd.Series(['site_2g', 'site_3g', 'site_4g', 'site_5g'])
ntechs = len(technologies)
nprovs = len(providers)

In [ ]:
# Create graph
df_extracted = extract_data(df, provider='Orange', techno='4g', region='Normandie')
G, pos = delaunay_graph(df_extracted)

In [ ]:
detected_cities = city_detection(extract_data(df, provider='Orange', techno='4g')[['x', 'y']])
cityness_proba = detected_cities['probas']
mean_distances = mean_distance_to_NN(extract_data(df, provider='Orange', techno='4g', region='Normandie')[['longitude', 'latitude']], n_neighbours=3) # 3 to have more neighbours

In [ ]:
G_dis = distance_criterion_enhanced(G, pos, cityness_proba=cityness_proba)
G_ang = angle_criterion_enhanced(G, pos, mean_distance_to_NN=mean_distances)
G_qua = quadrant_criterion_enhanced(G, pos)
G_tot = distance_criterion_enhanced(G, pos, cityness_proba=cityness_proba)
G_tot = angle_criterion_enhanced(G_tot, pos, mean_distance_to_NN=mean_distances)
G_tot = quadrant_criterion_enhanced(G_tot, pos)

In [ ]:
# Create a colormap
cmap = plt.cm.get_cmap('winter_r')  # 'cool' colormap for blue to gray
norm = mcolors.Normalize(vmin=0, vmax=1)

In [ ]:
def add_graph_edges(G: nx.Graph, pos: dict, fg: folium.FeatureGroup, colour: str):
    for edge in G.edges:
        stations = []
        stations.append(pos[edge[0]])
        stations.append(pos[edge[1]])
        stations = np.array(stations)

        folium.PolyLine(stations, color=colour, weight=2.5, opacity=1).add_to(fg)

In [ ]:
def marker_color(node: int):
    if(mean_distances[node] <= 1.0):
        return 'darkblue'
    elif((mean_distances[node] > 1) and (mean_distances[node] <= 2)):
        return 'azure'
    elif((mean_distances[node] > 2) and (mean_distances[node] <= 5)):
        return 'aquamarine'
    elif((mean_distances[node] > 5) and (mean_distances[node] <= 10)):
        return 'lime'
    else:
        return 'beige'

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=8.5, tiles="Cartodb Positron")
labels = folium.FeatureGroup("Labels", show=False).add_to(map)
edges = folium.FeatureGroup("Edges").add_to(map)
edges_dis = folium.FeatureGroup("Edges - distance", show=False).add_to(map)
edges_ang = folium.FeatureGroup("Edges - angle", show=False).add_to(map)
edges_qua = folium.FeatureGroup("Edges - quadrant", show=False).add_to(map)
edges_tot = folium.FeatureGroup("Edges - total").add_to(map)
points = folium.FeatureGroup("Points").add_to(map)

for edge in G.edges:
    stations = []
    stations.append(pos[edge[0]])
    stations.append(pos[edge[1]])
    stations = np.array(stations)
    
    folium.PolyLine(stations, color="lightgray", weight=2.5, opacity=1).add_to(edges)
    folium.map.Marker(
        (stations[0]+stations[1])/2,
        icon=folium.features.DivIcon(
            icon_size=(250,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 7pt">{km_distance(stations[0], stations[1]):.2f} km</div>'
        )
    ).add_to(labels)

add_graph_edges(G_dis, pos, edges_dis, colour="red")
add_graph_edges(G_ang, pos, edges_ang, colour="orange")
add_graph_edges(G_qua, pos, edges_qua, colour="green")
add_graph_edges(G_tot, pos, edges_tot, colour="goldenrod")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    color = marker_color(ind)
    points.add_child(folium.CircleMarker(location=[latitude, longitude], color=color, radius=5, popup=f"mean_dist={mean_distances.get(ind)}\nproba={cityness_proba.get(ind)}"))
folium.LayerControl().add_to(map)

map.save("../../out/maps/enhanced_neighborhood_default.html")